# Anthropic Claude + Gaugid Integration

This notebook demonstrates how to integrate Gaugid SDK with Anthropic Claude 4.5 to build a personalized chatbot.

In [ ]:
import os
import asyncio
from anthropic import Anthropic
from anthropic.types import MessageParam
from gaugid import GaugidClient

# Set your API keys
os.environ["ANTHROPIC_API_KEY"] = "your-key-here"
os.environ["GAUGID_CONNECTION_TOKEN"] = "your-token-here"

In [ ]:
async def load_user_context(client: GaugidClient) -> str:
    """Load user context from Gaugid profile using connection token mode."""
    # Connection token mode: No DID needed - profile resolved from token
    profile = await client.get_profile(
        scopes=["a2p:preferences", "a2p:professional", "a2p:context", "a2p:interests"]
    )
    
    # Extract memories from different memory types
    memories = profile.get("memories", {})
    episodic = memories.get("episodic", [])
    semantic = memories.get("semantic", [])
    procedural = memories.get("procedural", [])
    
    context_parts = []
    
    # Add episodic memories
    for memory in episodic:
        category = memory.get("category", "general")
        content = memory.get("content", "")
        context_parts.append(f"- [Episodic: {category}] {content}")
    
    # Add semantic memories
    for memory in semantic:
        category = memory.get("category", "general")
        content = memory.get("content", "")
        context_parts.append(f"- [Semantic: {category}] {content}")
    
    # Add procedural memories
    for memory in procedural:
        category = memory.get("category", "general")
        content = memory.get("content", "")
        context_parts.append(f"- [Procedural: {category}] {content}")
    
    return "\n".join(context_parts) if context_parts else "No user context available."

# Initialize clients
anthropic_client = Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])
gaugid_client = GaugidClient(connection_token=os.environ["GAUGID_CONNECTION_TOKEN"])

# Load user context (connection token mode - no DID needed)
user_context = await load_user_context(gaugid_client)
print("User context loaded:")
print(user_context)

In [ ]:
# Chat with Claude
messages: list[MessageParam] = []

user_input = "What are the best practices for async Python?"

messages.append({"role": "user", "content": user_input})

system_prompt = f"""You are a helpful AI assistant.

USER PROFILE (from Gaugid):
{user_context}

Adapt your responses to match user preferences and expertise level."""

response = anthropic_client.messages.create(
    model="claude-4-5",
    max_tokens=4096,
    system=system_prompt,
    messages=messages
)

print(f"User: {user_input}")
print(f"\nAssistant: {response.content[0].text}")

# Optionally, save the conversation as a memory
# Using connection token mode (no DID needed)
memory_result = await gaugid_client.propose_memory(
    content=f"User asked about async Python best practices",
    category="a2p:professional",
    memory_type="episodic",
    confidence=0.9,
    context="Conversation with Claude assistant"
)
print(f"\nMemory saved: {memory_result.get('proposal_id')}")